# Load dataset

In [1]:
import gdown
import os

url = 'https://drive.google.com/uc?id=1UazU8Dd1ZahFPKQL4bqgapfkqiZax6SO'
output = './l2r.tar.gz'

if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UazU8Dd1ZahFPKQL4bqgapfkqiZax6SO
To: /content/l2r.tar.gz
516MB [00:08, 60.3MB/s]


In [2]:
import tarfile

tar = tarfile.open(output, "r:gz")
tar.extractall()
tar.close()

In [3]:
from sklearn.datasets import load_svmlight_file

train_csr, train_y, train_qid = load_svmlight_file("l2r/train.txt.gz", query_id=True)

test_csr, test_y, test_qid = load_svmlight_file("l2r/test.txt.gz", query_id=True)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

In [4]:
train_csr[:2]

<2x699 sparse matrix of type '<class 'numpy.float64'>'
	with 373 stored elements in Compressed Sparse Row format>

In [87]:
train_y[15:25]

array([0., 1., 1., 1., 1., 2., 0., 1., 4., 1.])

In [88]:
train_qid[15:25]

array([4, 4, 4, 4, 4, 4, 5, 5, 5, 5])

In [10]:
# There are some invalid samples
import pandas as pd

docs_by_query = pd.DataFrame({'doc_index' : np.arange(len(train_y)), 'labels' : train_y, 'query' : train_qid}, index=train_qid)

good_indexes = []

for query in set(train_qid):
  try:
    if len(set(docs_by_query.loc[query].values[:, 1])) > 1:
      good_indexes.extend(docs_by_query.loc[query, 'doc_index'].values)
  except:
    continue
  
train_csr = train_csr[good_indexes]
train_qid = train_qid[good_indexes]
train_y = train_y[good_indexes]

# Model

In [69]:
import numpy as np

def max_dcg_score(doc_labels):
  t = np.sort(doc_labels)[::-1]
  return np.sum((np.power(2, t) - 1) / np.log2(np.arange(2, len(t) + 2)))


def dcg_score(scores, labels):
  documents = np.arange(len(labels))
  documents_order_according_to_scores = documents[(-scores).argsort()]
  return np.sum((np.power(2, labels[documents_order_according_to_scores]) - 1) / np.log2(documents + 2))


def group_by_ids(a, group_ids):
  a, ids = np.array(a), np.array(group_ids)
  id_bounds_idxs = [0]
  id_bounds_idxs.extend((ids[1:] != ids[:-1]).nonzero()[0] + 1)
  id_bounds_idxs.append(len(ids))
  a_layed_out = []
  for i in range(len(id_bounds_idxs)-1):
    a_layed_out.append(a[id_bounds_idxs[i] : id_bounds_idxs[i + 1]])
  return a_layed_out

In [73]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm


class LambdaMART:
  def __init__(self, n_trees=5, max_depth=8, learning_rate=0.5, dcg_k=-1):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.learning_rate = learning_rate
    self.trees = []
    # self.k = dcg_k
 
  def fit(self, X, y, query_ids):
    assert X.shape[0] == len(y)
    n_samples = X.shape[0]

    y_by_query = group_by_ids(y, query_ids)
    model_scores_by_query = [np.zeros(len(scores)) for scores in y_by_query]
    max_dcg_by_query = [max_dcg_score(scores) for scores in y_by_query]
    # max_dcg_at_k(scores, self.dcg_k)
    
    for k in tqdm(range(self.n_trees)):
      lambdas, w = np.zeros(n_samples), np.zeros(n_samples)
      doc_idx = 0
      
      for y, model_scores, max_DCG in zip(y_by_query, model_scores_by_query, max_dcg_by_query):
        n_docs = len(y)
        doc_ranks_predicted = np.zeros(n_docs, dtype=np.int64) 
        doc_ranks_predicted[(-model_scores).argsort()] = np.arange(n_docs)

        for y_i, s_i, rank_i in zip(y, model_scores, doc_ranks_predicted):
          indices_j = (y != y_i)
          y_j, s_j, rank_j = y[indices_j], model_scores[indices_j], doc_ranks_predicted[indices_j]

          delta_DCG = np.abs(
              (np.power(2, y_i) - np.power(2, y_j)) * 
              (1. / np.log2(rank_i + 2.) - 1. / np.log2(rank_j + 2.))
          )
          rho_i_j = 1. / (1. + np.exp(np.abs(s_i - s_j)))
          lambda_i_j = -rho_i_j * delta_DCG

          lambda_i = (np.sign(y_i - y_j) * lambda_i_j).sum() / max_DCG
          w_i = (rho_i_j * (1 - rho_i_j) * delta_DCG).sum() / max_DCG

          lambdas[doc_idx], w[doc_idx] = lambda_i, w_i
          doc_idx += 1
      
      tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_leaf=10)
      tree.fit(X, lambdas)
 
      model_scores = np.concatenate(model_scores_by_query)
      leaf_by_doc_index = tree.apply(X)

      for leaf in set(leaf_by_doc_index):
        one_leaf_docs_indices = np.where(leaf_by_doc_index == leaf)[0]
        gamma_l_k = lambdas[one_leaf_docs_indices].sum() / w[one_leaf_docs_indices].sum()
        tree.tree_.value[leaf] = -gamma_l_k * self.learning_rate
        model_scores[one_leaf_docs_indices] -= gamma_l_k * self.learning_rate
      
      model_scores_by_query = group_by_ids(model_scores, query_ids)
      
      self.trees.append(tree)
  
  def predict(self, X):
    model_scores = np.zeros(X.shape[0])
    for tree in self.trees:
      model_scores += tree.predict(X)
    return model_scores

# Training

In [71]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_idx, val_idx = next(gss.split(train_csr, train_y, train_qid))

X_train, y_train, qid_train = train_csr[train_idx], train_y[train_idx], train_qid[train_idx]

X_val, y_val, qid_val = train_csr[val_idx], train_y[val_idx], train_qid[val_idx]

In [74]:
model = LambdaMART(5, 3, 0.2)
model.fit(X_train, y_train, qid_train)

100%|██████████| 5/5 [03:20<00:00, 40.02s/it]


In [75]:
def get_ndcg_score(scores, labels, query_ids):
  scores_grouped = group_by_ids(scores, query_ids)
  labels_grouped = group_by_ids(labels, query_ids)

  dcg = np.array([dcg_score(scores, labels) for scores, labels in zip(scores_grouped, labels_grouped)])
  max_dcg = np.array([max_dcg_score(labels) for labels in labels_grouped])

  ndcg = val_dcg / val_max_dcg

  return np.mean(ndcg)

In [76]:
pred = model.predict(X_val)

In [77]:
get_ndcg_score(pred, y_val, qid_val)

0.8039031198639267

# Submit

In [ ]:
def order_docs(doc_scores, qids):
    assert len(doc_scores) == len(qids)
    doc_ids = np.arange(1, len(doc_scores) + 1)
    ordered_docs = np.zeros(len(doc_scores), dtype=np.int32)
    
    qid_prev = qids[0]
    i_prev = 0

    for i, qid_i in enumerate(qids):
        if qid_i != qid_prev:
            ordered_docs[i_prev:i] = doc_ids[np.argsort(-doc_scores[i_prev:i]) + i_prev]
            i_prev = i
            qid_prev = qid_i
    ordered_docs[i_prev:] = doc_ids[np.argsort(-doc_scores[i_prev:]) + i_prev]
    
    return ordered_docs

In [81]:
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [84]:
import kaggle

pred = model.predict(test_csr)

sample = pd.read_csv('l2r/sample.made.fall.2019')
docs = order_docs(pred, sample['QueryId'].values)
sample['DocumentId'] = docs
sample.to_csv('submission.txt', index=False)

kaggle.api.competition_submit('submission.txt', '', 'learning-to-rank-made-fall-2019')

100%|██████████| 2.83M/2.83M [00:10<00:00, 291kB/s]


Successfully submitted to Learning to rank MADE Fall 2019